# <center> INTRODUCTION À PYTHON POUR L'ÉCONOMIE APPLIQUÉE</center>
## <center> COURS 0 : </center>
## <center> QUELQUES EXEMPLES INTRODUCTIFS </center>
#### <center>Michal Urdanivia (UGA)</center>
#### <center> michal.wong-urdanivia@univ-grenoble-alpes.fr </center>

### <center> PRÉSENTATION </center>

Dans cette introduction nous allons donner un très court aperçu des possibilités d'utilisation du langage Python pour l'économiste. Pour cela:
- nous allons présenter des exemples de simulation de modèles simples de l'économie,
- des exemples du travail empirique pouvant être fait.

**Résumé:** Un notebook Jupyter est un document composé de **cellules** qui contiennent:
- Le code de programmation qui dans notre cas sera du code Python.
- Du texte dont du texte mathématiques écrit dans le "style" de LaTex. Ces cellules sont des cellules *markdown*.
- Des résultats issus de l'execution du code programmé.

Ainsi, ceci est une cellule avec du texte et plus précisément cellule *markdown*.

On peut y écrire sous forme de listes:

1. **Première** entrée.
2. *Deuxième* entrée.
3. ~~Troisième~~ item entrée barrée.

On peut écrire de maths, e.g. $\alpha^2$ ou

$$
X = \int_0^{\infty} \frac{x}{x+1} \mathrm{d}x
$$

In [ ]:
# Ceci est une cellule de code
x = 10   # Ce qui est écrit après un "#" est du "commentaire" du code, il n'est donc pas exécuté

# Procédons à des calculs
a = 20
b = 30
c = a + b

# Affichons les résultats (au-dessous de cette cellule)
print(c)

On peut comme ici écrire un peu plus de texte(cellule *markdown*) et poursuivre les calculs(cellules de code).

In [ ]:
d = c * 2
print(d)
print(c)

**Remarque:** 

Bien que JupyterLab/Jupyter s'execute avec un serveur, cela se fait sans besoin de connexion web (le chemin étant quelque chose comme *localhost:8888/lab*).<br>

**Binder:** Une exception est d'utiliser [binder](https://mybinder.org), dans ce cas JupyterLab s'executera sur un cloud.

### <center> PROBLÉME CLASSIQUE DU CONSOMMATEUR </center>

On considère ici l'exemple classique des manuels de cours de Microéconomie que vous avez vu dans vos études.

Le problème est le suivant:

$$
\begin{align*}
V(p_{1},p_{2},I) & = \max_{q_{1},q_{2}} q_{1}^{\alpha}q_{2}^{1-\alpha}\\
 & \text{s.c.}\\
p_{1}q_{1}+p_{2}q_{2} & \leq R, \quad p_{1}, p_{2}, R > 0, \quad q_{1},q_{2}  \geq 0
\end{align*}
$$

On peut résoudre ce problème **numériquement**.

1. Choix de **paramètres**:

In [ ]:
alpha = 0.5
R = 10
p1 = 1
p2 = 2

2. La **fonction objectif** du consommateur est:

In [ ]:
# On définit la fonction objectif.
def val_cons(q1, alpha, R, p1, p2): 
    
    # a. Tout le revenu n'est pas dépensé sur le premier bien
    #    Et ce qui est dépensé dans le deuxième est:
    q2 = (R - p1*q1) / p2 
    
    # b. L'utilité obtenue est:
    util = q1**alpha*q2**(1-alpha)
    
    return util # résultat produit par la fonction

3. On utilise une fonction du module *scipy* pour **résoudre le problème**.

In [ ]:
# a. On importe à partir du module scipy la fonction optimize
from scipy import optimize

# b. la valeur du choix comme seulement fonction de q1
obj = lambda q1: -val_cons(q1, alpha, R, p1, p2)

# c. on appelle la fonction minimisatrice
solution = optimize.minimize_scalar(obj, bounds=(0, R/p1))

# d. affichage des résultats
q1 = solution.x
q2 = (R - q1*p1) / p2
print(q1, q2)

**Tâche**: résoudre le même problème avec la fonction d'utilité CES.

$$
u(q_1,q_2) = (\alpha q_1^{-\beta} + (1-\alpha)q_2^{-\beta})^{-1/\beta}
$$

In [ ]:
# a. choix des paramètres
alpha = 0.5
beta = 0.000001
R = 10
p1 = 1
p2 = 2

# b. valeur du choix
def val_cons_ces(q1, alpha, beta, R, p1, p2):
    q2 = (R - p1*q1)/p2
    if q1 > 0 and q2 > 0:
        util = (alpha*q1**(-beta) + (1 - alpha)*q2**(-beta))**(-1 / beta) 
    else:
        util = 0
    return util

# c. objectif
obj = lambda q1: -val_cons_ces(q1, alpha, beta, R, p1, p2)

# d. solution
solution = optimize.minimize_scalar(obj, bounds=(0, R/p1))

# e. résultat
q1 = solution.x
q2 = (R - q1 * p1)/p2
print(q1, q2)

### <center> SIMULATION DU MODÈLE AS-AD </center>

**Remarque**: c'est un modèle macroéconomique pour [l'offre et la demande globales dans une économie](https://fr.wikipedia.org/wiki/Mod%C3%A8le_OG-DG).

Soit le modèle défini par:

$$
\begin{align*}
\hat{y}_{t} &= b\hat{y}_{t - 1}+\beta(z_{t} - z_{t-1}) -a\beta s_{t} + a\beta\phi s_{t-1} \\
\hat{\pi}_{t} &= b\hat{\pi}_{t - 1}+\beta\gamma z_{t} - \beta\phi\gamma z_{t}+\beta s_{t} - \beta\phi s_{t-1} \\
z_{t} &= \delta z_{t - 1}+x_{t}, x_{t} \sim N(0, \sigma_x^2) \\
s_{t} &= \omega s_{t - 1}+c_{t}, c_{t} \sim N(0, \sigma_c^2) \\
b &= \frac{1 + a\phi\gamma}{1 + a\gamma} \\
\beta &= \frac{1}{1 + a\gamma}
\end{align*}
$$

où $\hat{y}_{t}$ est l'écart de production, $\hat{\pi}_{t}$ est l'écart d'inflation, $z_{t}$ est un choc de demande AR(1), et $\hat{s}_{t}$ est un chox d'offre AR(1).

1. Choix de **paramètres**:

In [ ]:
a = 0.4
gamma = 0.1
phi = 0.9
delta = 0.8
omega = 0.15
sigma_x = 1
sigma_c = 0.4
T = 100

2. Calcul des **paramètres composites**:

In [ ]:
b = (1 + a*phi*gamma)/(1 + a*gamma)
beta = 1/(1 + a*gamma) 

3. Définition des **fonctions du modèle**:

In [ ]:
y_hat_func = lambda y_hat_lag, z,z_lag, s, s_lag: b*y_hat_lag + beta*(z-z_lag) - a*beta*s + a*beta*phi*s_lag
pi_hat_func = lambda pi_lag,z,z_lag,s,s_lag: b*pi_lag + beta*gamma*z - beta*phi*gamma*z_lag + beta*s - beta*phi*s_lag
z_func = lambda z_lag,x: delta*z_lag + x
s_func = lambda s_lag,c: omega*s_lag + c

4. On execute la **simulation**:

In [ ]:
import numpy as np # importation du module numpy

# a. mise en place du générateur de nombre aléatoires
np.random.seed(2015)   

# b. affectation des données simulées
x = np.random.normal(loc=0, scale=sigma_x, size=T)
c = np.random.normal(loc=0, scale=sigma_c, size=T)
z = np.zeros(T)
s = np.zeros(T)
y_hat = np.zeros(T)
pi_hat = np.zeros(T)

# c. execution de la simulation
for t in range(1,T):

    # i. actualisation de z et s
    z[t] = z_func(z[t-1], x[t])
    s[t] = s_func(s[t-1], c[t])

    # ii. calcul de y og pi 
    y_hat[t] = y_hat_func(y_hat[t-1], z[t], z[t-1], s[t], s[t-1])
    pi_hat[t] = pi_hat_func(pi_hat[t-1], z[t], z[t-1], s[t], s[t-1])

5. **Visualisation** de la simulation:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.plot(y_hat, label='$\\hat{y}$')
ax.plot(pi_hat, label='$\\hat{\pi}$')

ax.set_xlabel('temps')

ax.set_ylabel('pourcentage')
ax.set_ylim([-8,8])

ax.legend(loc='upper left');

On peut aussi utiliser **le style seaborn**:

In [ ]:
plt.style.use('seaborn-whitegrid')

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.plot(y_hat,label='$\\hat{y}$')
ax.plot(pi_hat,label='$\\hat{pi}$')

ax.set_xlabel('temps')

ax.set_ylabel('poucentage')
ax.set_ylim([-8,8])

ax.legend(loc='upper left',facecolor='white',frameon='True');

### <center> MODULES</center>

Un **module** est un fichier **.py**-file avec des fonctions qu'on importe et auxquelle on peut alors faire appel dans le notebook.

Vous pouvez ouvrir **monmodule.py** et y jeter un oeil.

In [1]:
import monmodule

In [4]:
x = 5
y = monmodule.mafonction(x)
print(y)

25


# Analyse de données

## Enquête Emploi de 2012

On utilise un extrait des données de l'enquête emploi de l'Insee pour 2012 pour mesurer l'écart salarial entre femmes et hommes.

In [7]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np

In [3]:
eemploi = pd.read_csv('/Users/michalurdanivia/Documents/GitHub/Cours-UGA-Python-L3-Eco'\
    '/Autres/Donnees/Enquete_Emploi_2012/eemploi2012_s0.csv', low_memory=False) 
eemploi = eemploi[['ACTEU', 'SEXE', 'DDIPL', 'FORDAT', 'MATRI', 'TYPMEN5', 'ZUS', 'REG','AG', 
             'AGQ', 'AGEQ','AGE','AG5','NBENF3', 'NBENF6', 'NBENF18', 'SALRED', 'NBHEUR', 
             'NATPERC','NATMERC', 'PAIPERC','PAIMERC','TRIM', 'RGI', 'IDENT', 'NOI', 
             'DIP', 'DIP11','CONTRA','CSER', 'CSPM', 'CSPP', 'ANNEE', 'CHPUB', 'NAFG4N', 
             'ANCENTR4', 'exper', 'adfe', 'lsalhor']]

eemploi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31045 entries, 0 to 31044
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ACTEU     31045 non-null  int64  
 1   SEXE      31045 non-null  int64  
 2   DDIPL     31045 non-null  int64  
 3   FORDAT    31045 non-null  int64  
 4   MATRI     31045 non-null  int64  
 5   TYPMEN5   31045 non-null  int64  
 6   ZUS       31045 non-null  int64  
 7   REG       31045 non-null  int64  
 8   AG        31045 non-null  int64  
 9   AGQ       31045 non-null  int64  
 10  AGEQ      31045 non-null  int64  
 11  AGE       31045 non-null  int64  
 12  AG5       31045 non-null  int64  
 13  NBENF3    31045 non-null  int64  
 14  NBENF6    31045 non-null  int64  
 15  NBENF18   31045 non-null  int64  
 16  SALRED    19764 non-null  float64
 17  NBHEUR    19764 non-null  float64
 18  NATPERC   31045 non-null  int64  
 19  NATMERC   31045 non-null  int64  
 20  PAIPERC   31045 non-null  in

In [8]:
df = eemploi[(eemploi.ACTEU == 1) & (eemploi.NAFG4N !='00') & (eemploi.CSER !=0)]
df = df.dropna()
df['lsal'] = np.log(df['SALRED']/df['NBHEUR']
df['afe'] = df['FORDAT'] - (2012 - df['AG'])
df['exp'] = df['AG'] - df['afe']
dummies = df[['DDIPL', 'MATRI', 'REG', 'ZUS', 'CSER', 'NAFG4N', 'ANCENTR4', 'SEXE']]
dummies = pd.get_dummies(data = dummies, columns=['DDIPL', 'MATRI', 'REG', 'ZUS', 'CSER', 'NAFG4N', 'ANCENTR4', 'SEXE'])
df = pd.concat([df, dummies], axis = 1)
listvar = ['lsal', 'exp'] + list(dummies)
df[listvar].describe()

SyntaxError: invalid syntax (<ipython-input-8-5bbe189cecc2>, line 4)

In [ ]:
lsal_tab = df.groupby(["DDIPL", "SEXE", "CSER"])['lsal'].mean().unstack(level="SEXE")
lsal_tab

In [ ]:
import patsy
formula = """
lsal ~ SEXE_2
       + (C(DDIPL) + C(ANCENTR4) + C(NAFG4N) + C(REG) + C(CSER) + C(MATRI))**2 
       + exp + I(exp**2) + I(exp**3) + I(exp**4)      
"""
Y, X = patsy.dmatrices(formula, df, return_type = "dataframe")

In [ ]:
linreg_ols0 = sm.OLS(endog = Y, exog = X[['Intercept','SEXE_2']], missing = 'drop').fit(cov_type='HC0')
linreg_ols1 = sm.OLS(endog = Y, exog = X, missing = 'drop').fit(cov_type='HC0')
from stargazer.stargazer import Stargazer, LineLocation
stargazer = Stargazer([linreg_ols0 , linreg_ols1])
stargazer.covariate_order(['SEXE_2'])
stargazer